# StackOverflow : Tag Prediction

# 1. Buisness Problem

# 1.1 Description

**Description**

Stack Overflow is the largest, most trusted online community for developers to learn, share their programming knowledge, and build their careers.

Stack Overflow is something which every programmer use one way or another. Each month, over 50 million developers come to Stack Overflow to learn, share their knowledge, and build their careers. It features questions and answers on a wide range of topics in computer programming. The website serves as a platform for users to ask and answer questions, and, through membership and active participation, to vote questions and answers up or down and edit questions and answers in a fashion similar to a wiki or Digg. As of April 2014 Stack Overflow has over 4,000,000 registered users, and it exceeded 10,000,000 questions in late August 2015. Based on the type of tags assigned to questions, the top eight most discussed topics on the site are: Java, JavaScript, C#, PHP, Android, jQuery, Python and HTML.

**Problem Statemtent**

Suggest the tags based on the content that was there in the question posted on Stackoverflow.

**Source:** https://www.kaggle.com/c/facebook-recruiting-iii-keyword-extraction/

# 1.2 Source / useful links
Data Source : https://www.kaggle.com/c/facebook-recruiting-iii-keyword-extraction/data 

Youtube : https://youtu.be/nNDqbUhtIRg 

Research paper : https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/tagging-1.pdf 

Research paper : https://dl.acm.org/citation.cfm?id=2660970&dl=ACM&coll=DL

# 1.3 Real World / Business Objectives and Constraints






1. Predict as many tags as possible with high precision and recall.

2. Incorrect tags could impact customer experience on StackOverflow.

3. No strict latency constraints.

# 2. Machine Learning problem

# 2.1 Data
**2.1.1 Data Overview**

Refer: https://www.kaggle.com/c/facebook-recruiting-iii-keyword-extraction/data 
All of the data is in 2 files: Train and Test.

Train.csv contains 4 columns: Id,Title,Body,Tags.

Test.csv contains the same columns but without the Tags, which you are to predict.

Size of Train.csv - 6.75GB

Size of Test.csv - 2GB

Number of rows in Train.csv = 6034195

The questions are randomized and contains a mix of verbose text sites as well as sites related to math and programming. The number of questions from each site may vary, and no filtering has been performed on the questions (such as closed questions).


**Note :-** Due to my PC constraints having 4 GB of RAM only i randomly sampled 100k rows from Train.csv and saved as Train_data.csv and i will be using the same.

**Data Field Explaination**

Dataset contains 100k rows. The columns in the table are:

Id - Unique identifier for each question

Title - The question's title

Body - The body of the question

Tags - The tags associated with the question in a space-seperated format (all lowercase, should not contain tabs '\t' or ampersands '&')



# 2.2 Mapping the real-world problem to a Machine Learning Problem

# 2.2.1 Type of Machine Learning Problem

It is a multi-label classification problem 
Multi-label Classification: Multilabel classification assigns to each sample a set of target labels. This can be thought as predicting properties of a data-point that are not mutually exclusive, such as topics that are relevant for a document. A question on Stackoverflow might be about any of C, Pointers, FileIO and/or memory-management at the same time or none of these. 
Credit: http://scikit-learn.org/stable/modules/multiclass.html


**2.2.2 Performance metric**

**Micro-Averaged F1-Score (Mean F Score) :**   The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. The formula for the F1 score is:

F1 = 2 (precision recall) / (precision + recall)

In the multi-class and multi-label case, this is the weighted average of the F1 score of each class. 

**'Micro f1 score': **

Calculate metrics globally by counting the total true positives, false negatives and false positives. This is a better metric when we have class imbalance. 

**'Macro f1 score': **

Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account. 

https://www.kaggle.com/wiki/MeanFScore 
http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html 

**Hamming loss :**  The Hamming loss is the fraction of labels that are incorrectly predicted. 
https://www.kaggle.com/wiki/HammingLoss 

# 3. Exploratory Data Analysis

# 3.1 Data Loading 



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp "/content/drive/My Drive/StackOverflow/Train_data.csv" "Train_data.csv"

In [0]:
import pandas as pd

In [0]:
data=pd.read_csv("Train_data.csv")

In [0]:
data.head()

,Unnamed: 0,Id,Title,Body,Tags
0,2221128,2221129,Linearization for differential equation,<p>I'm trying to make linearization for the fo...,differential-equations
1,1757257,1757258,Java upload file (image specifically) from a J...,<p>I am going mad trying to find an example of...,java asp.net-mvc file-upload applet jnlp
2,1497316,1497317,how to set separate css to each menu item in t...,<p>i have a menu control like below</p>\n\n<p>...,c# asp.net
3,3055821,3055822,jQuery multiple event handler,<p>What does the keyup() at the end of it mean...,jquery
4,4281363,4281364,Applications for using couchDB and a RDBMS tog...,<p>Wondering if there was a scenario where one...,database architecture couchdb rdbms


****3.1.1 Using Pandas with SQLite to Load the data****

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import datetime
import sqlite3
import sqlalchemy

In [0]:
#Creating db file from csv
if not os.path.isfile('Train_data.db'):
  engine= sqlalchemy.create_engine('sqlite:///Train_data.db')
  start=datetime.now()
  Csize=10000
  j=0
  index_start=1
  for df in pd.read_csv('Train_data.csv',names=['Id','Title','Body','Tags'], chunksize = Csize,iterator=True,encoding='utf-8'):
    df.index+=index_start
    j+=1
    print('{} rows'.format(j*Csize))
    df.to_sql('data',engine,if_exists='append')
    index_start=df.index[-1]+1
    print("Time taken :" ,datetime.now() - start)
  


10000 rows
Time taken : 0:00:00.924800
20000 rows
Time taken : 0:00:01.836806
30000 rows
Time taken : 0:00:02.735917
40000 rows
Time taken : 0:00:03.717769
50000 rows
Time taken : 0:00:04.622163
60000 rows
Time taken : 0:00:05.524799
70000 rows
Time taken : 0:00:06.417541
80000 rows
Time taken : 0:00:07.282396
90000 rows
Time taken : 0:00:08.221197
100000 rows
Time taken : 0:00:09.098211
110000 rows
Time taken : 0:00:09.114596


**3.1.2 Counting the number of rows**

In [0]:
if os.path.isfile('Train_data.db'):
  con=sqlite3.connect('Train_data.db')
  n_rows=pd.read_sql(""" SELECT count(*) FROM `data`""",con)
  print("No of rows : ",n_rows['count(*)'].values[0])
  con.close()

No of rows :  100001


**3.1.3 Checking for duplicates**

In [0]:
con=sqlite3.connect('Train_data.db')
query="SELECT `Title`,`Tags`,`Body`, COUNT(*) as cnt_dup FROM `data` GROUP BY `Title`,`Body`,`Tags`"
dataframe=pd.read_sql(query,con)
con.close()


In [0]:
df_puplicates.head()

,Title,Tags,Body,cnt_dup
0,Advise on tackling syntax errors in Java,java,"<p>I very inexperience with Java, I'm spending...",1
1,Controlling the Lego WeDo Device,c# api lego,<p>Has anyone written a API for the Lego WeDo ...,1
2,Count inverse document frequency (idf) in eve...,php,"<p>I wanna count idf, the formula is <code>IDF...",1
3,"FrameView with ""flip"" animation images not sc...",android image layout resize scaling,<p>Can't seem to work out what I've got wrong....,1
4,Going from Java to Classic ASP / VB,vb6 career-development legacy-code,"<p>Right now, I work on a small team at a larg...",1


In [0]:
n_row=n_rows['count(*)'].values[0]
n_row_dup=dataframe.shape[0]
n_dup=n_row-n_row_dup
print(n_row)
print(n_row_dup)
print("No of Duplicates= ",n_dup)
print("% duplication = ",(n_dup/n_row)*100)

100001
99462
No of Duplicates=  539
% duplication =  0.5389946100538995


In [0]:
dataframe.cnt_dup.value_counts()

1    98923
2      539
Name: cnt_dup, dtype: int64

**Observation :** 

1.No of question one time= 98923

2.No of question two time= 539

In [0]:
#Adding tag_count in dataframe
dataframe['tag_count']=dataframe['Tags'].apply(lambda text : len(text.split(" ")))
dataframe.head()

,Title,Tags,Body,cnt_dup,tag_count
0,Advise on tackling syntax errors in Java,java,"<p>I very inexperience with Java, I'm spending...",1,1
1,Controlling the Lego WeDo Device,c# api lego,<p>Has anyone written a API for the Lego WeDo ...,1,3
2,Count inverse document frequency (idf) in eve...,php,"<p>I wanna count idf, the formula is <code>IDF...",1,1
3,"FrameView with ""flip"" animation images not sc...",android image layout resize scaling,<p>Can't seem to work out what I've got wrong....,1,5
4,Going from Java to Classic ASP / VB,vb6 career-development legacy-code,"<p>Right now, I work on a small team at a larg...",1,3


In [0]:
dataframe['tag_count'].value_counts()

3    28412
2    26520
4    19206
1    13682
5    11642
Name: tag_count, dtype: int64

**Observation :**

1.No of questios having 3 tags =28412

2.No of questios having 2 tags =26520

3.No of questios having 4 tags =19206

4.No of questios having 1 tags =13682

5.No of questios having 5 tags =11642

Order a/c to no of question :

3tags>2 tags > 4 tags >1 tag >5 tags
